# Creating Word2Vec Embeddings

We will use sentences from periodika to create a word2vec model

In [1]:
# TODO load chunked data into a single dataframe
# TODO install gensim if it is not already installed
# TODO load lemma data in a format suitable for Word2Vec

# first check python version
import sys
print(f"Python version: {sys.version}")
import pickle

# date
from datetime import datetime
now = datetime.now()
print(f"Date: {now}")

# we will use Path from pathlib to work with paths
from pathlib import Path

# we will use tqdm for progress bars
from tqdm import tqdm

## We will be loading all parquet files from a parquet folder into a single pandas dataframe

## then we will calculate word richness for all novels and plot it using Plotly
# first install Pandas if needed using pip and optional dependencies
# https://pandas.pydata.org/pandas-docs/stable/getting_started/install.html#install-optional-dependencies

# pip install "pandas[performance, plot, output_formatting, computation, excel, html, parquet, compression, consortium-standard]"

# try to import pandas
try:
    import pandas as pd
    print(f"pandas version: {pd.__version__}")
except ImportError:
    print("pandas not found")
    raise ImportError

# issue with scipy library removing triu function
# https://stackoverflow.com/questions/78279136/importerror-cannot-import-name-triu-from-scipy-linalg-when-importing-gens
# https://github.com/piskvorky/gensim/issues/3525
try:
    import gensim
    print(f"gensim version: {gensim.__version__}")
except ImportError:
    print("gensim not found")
    raise ImportError

Python version: 3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]
Date: 2025-02-25 10:11:06.638636
pandas version: 2.2.1
gensim version: 4.3.2


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import gensim.models

## Load and prepare parquet from latsenrom

In [4]:
src = Path("../../lnb_lat_sen_rom/parquet/latsenrom_2025_02_05.parquet")
# assert file exists
if src.exists():
    print(f"File {src} exists")
else:
    print(f"File does not exist")


File ..\..\lnb_lat_sen_rom\parquet\latsenrom_2025_02_05.parquet exists


In [5]:
# let's load df
df = pd.read_parquet(src)
# shape
print(f"Shape: {df.shape}")
# head
df.head()

Shape: (37605476, 17)


,deprel,form,index,lemma,parent,pos,tag,ufeats,upos,sent_ndx,author,title,dom_id,file_stem,file_stem_short,firstEdition,term
0,nmod,Mīlas,1,mīla,2.0,ncfsg_,ncfsg4,Case=Gen|Gender=Fem|Number=Sing,NOUN,0,AizsV,MilaU,1049452,AizsV_MilaU_1049452,AizsV_MilaU,1933,mīla
1,nmod,ārprāta,2,ārprāts,3.0,ncmsg_,ncmsg1,Case=Gen|Gender=Masc|Number=Sing,NOUN,0,AizsV,MilaU,1049452,AizsV_MilaU_1049452,AizsV_MilaU,1933,ārprāts
2,obl,varā,3,vara,6.0,ncfsl_,ncfsl4,Case=Loc|Gender=Fem|Number=Sing,NOUN,0,AizsV,MilaU,1049452,AizsV_MilaU_1049452,AizsV_MilaU,1933,vara
3,nmod,ROMĀNS,4,Romāns,6.0,npmsn_,npmsn1,Case=Nom|Gender=Masc|Number=Sing,PROPN,0,AizsV,MilaU,1049452,AizsV_MilaU_1049452,AizsV_MilaU,1933,Romāns
4,punct,„,5,"""",6.0,zq,zq,_,PUNCT,0,AizsV,MilaU,1049452,AizsV_MilaU_1049452,AizsV_MilaU,1933,""""


In [6]:
# how many unique file_stem
print(f"Unique file_stem: {df.file_stem.nunique()}")

Unique file_stem: 485


In [7]:
# now we need to create a list of sentences from the lemma column
# we will group by file_stem and sent_ndx and then join the lemmas with a space
# we will use tqdm to show progress bar

# first we will group by file_stem and sent_ndx
grouped = df.groupby(["file_stem", "sent_ndx"])["lemma"].apply(list)
# let's check the first 5 rows
grouped.head()


file_stem            sent_ndx
AizsV_MilaU_1049452  0           [mīla, ārprāts, vara, Romāns, ", redzēt, ,, Ve...
                     1           [Līdija, pieliekties, un, nebēdīgi, uzšļākt, V...
                     2           [nu, tu, savs, sodīt, izbēgt, !, ", likties, ,...
                     3           [Vents, piedraudēt, un, satvert, viņa, roka, t...
                     4           [nekas, nebij, !, ", zināt, zinātnieks, rādīt,...
Name: lemma, dtype: object

In [8]:
# create a datafame from the grouped series that also has a sentence_length column
sentences_df = grouped.reset_index(name="sentences")
# add sentence_length
sentences_df["sentence_length"] = sentences.sentences.apply(len)
# check first 5 rows
sentences_df.head()

,file_stem,sent_ndx,sentences,sentence_length
0,AizsV_MilaU_1049452,0,"[mīla, ārprāts, vara, Romāns, "", redzēt, ,, Ve...",34
1,AizsV_MilaU_1049452,1,"[Līdija, pieliekties, un, nebēdīgi, uzšļākt, V...",34
2,AizsV_MilaU_1049452,2,"[nu, tu, savs, sodīt, izbēgt, !, "", likties, ,...",28
3,AizsV_MilaU_1049452,3,"[Vents, piedraudēt, un, satvert, viņa, roka, t...",35
4,AizsV_MilaU_1049452,4,"[nekas, nebij, !, "", zināt, zinātnieks, rādīt,...",34


In [9]:
# describe sentence_length
sentences_df.sentence_length.describe()

count    680143.000000
mean         55.290543
std          46.876084
min           1.000000
25%          25.000000
50%          43.000000
75%          70.000000
max         671.000000
Name: sentence_length, dtype: float64

## Save and Load sentences for word2vec parquet

In [4]:
# save sentences to parquet
sentences_dst = Path("../parquet/sentences_for_word2vec.parquet")
# sentences_df.to_parquet(sentences_dst, index=False, compression="gzip")
sentences_df = pd.read_parquet(sentences_dst)
# shape
print(f"Shape: {sentences_df.shape}")
# head
sentences_df.head()

Shape: (680143, 4)


,file_stem,sent_ndx,sentences,sentence_length
0,AizsV_MilaU_1049452,0,"[mīla, ārprāts, vara, Romāns, "", redzēt, ,, Ve...",34
1,AizsV_MilaU_1049452,1,"[Līdija, pieliekties, un, nebēdīgi, uzšļākt, V...",34
2,AizsV_MilaU_1049452,2,"[nu, tu, savs, sodīt, izbēgt, !, "", likties, ,...",28
3,AizsV_MilaU_1049452,3,"[Vents, piedraudēt, un, satvert, viņa, roka, t...",35
4,AizsV_MilaU_1049452,4,"[nekas, nebij, !, "", zināt, zinātnieks, rādīt,...",34


In [11]:
# sentences_df = sentences


In [5]:
# create a list of sentences by joining the lemmas with a space 
# sentences = sentences_df.sentences.apply(lambda x: " ".join(x)).tolist()
sentences = sentences_df.sentences.tolist()
# how many sentences?
print(f"Number of sentences: {len(sentences)}")

Number of sentences: 680143


In [12]:
# save sentences to pickle
sentences_pickle = Path("../pickle/sentences_for_word2vec_2025_02_07.pickle")
with open(sentences_pickle, "wb") as f:
    pickle.dump(sentences, f)
# print resulting file size
print(f"File size: {sentences_pickle.stat().st_size}")

File size: 128703282


In [15]:
# first 3 sentences
print(sentences[:3])

[['mīla', 'ārprāts', 'vara', 'Romāns', 'redzēt', 'Vents', 'cik', 'līksmi', 'griezties', 'dzirnaviņas', 'spārns', 'kā', 'zelts', 'taurenītis', 'mīla', 'rotaļa', 'viņš', 'nostatīt', 'tāļredze', 'spogulis', 'acs', 'Vents', 'kļūt', 'tā', 'žēl'], ['Līdija', 'pieliekties', 'un', 'nebēdīgi', 'uzšļākt', 'Vents', 'šalts', 'vēss', 'ūdens', 'desmit', 'kilometrs', 'attāļums', 'no', 'Bornholma', 'ārprātīgs', 'orkāns', 'sirmot', 'jūra', 'dzelme', 'viņš', 'pateikties', 'dievs', 'kas', 'būt', 'ļaut', 'nogalināt', 'šī', 'nelaimīgs', 'ģēnijs'], ['nu', 'tu', 'savs', 'sodīt', 'izbēgt', 'likties', 'ka', 'milzīgs', 'vilnis', 'skārta', 'padebesis', 'tik', 'augstu', 'lidot', 'vilnis', 'strūkla', 'ārprātīgs', 'ciešana', 'atspoguļoties', 'ikviens', 'Ādlers', 'vaibsts']]


In [17]:
# save sentences as text with each sentence on a new line
# then save it in a compressed zip file
sentences_txt = Path("../txt/sentences_for_word2vec_2025_02_07.txt")
with open(sentences_txt, "w", encoding="utf-8") as f:
    f.write("\n".join((" ".join(row_list) for row_list in sentences)))
# print resulting file size
print(f"File size: {sentences_txt.stat().st_size}")

# now compress the file using zip

import zipfile
zipfile_dst = Path("../zip/sentences_for_word2vec_2025_02_07.zip")
with zipfile.ZipFile(zipfile_dst, "w", compression=zipfile.ZIP_DEFLATED) as z:
    z.write(sentences_txt)

# print resulting file size
print(f"File size: {zipfile_dst.stat().st_size}")


File size: 194258734
File size: 70280009


In [6]:
# let's get rid of any elements that are punctuation
import string
# let's iterate over sentences and remove punctuation
sentences = [[word for word in sentence if word not in string.punctuation] for sentence in sentences]
# how many sentences?
print(f"Number of sentences: {len(sentences)}")

Number of sentences: 680143


In [7]:
# remove sentences that have less than 2 words
sentences = [sentence for sentence in sentences if len(sentence) > 1]
# how many sentences?
print(f"Number of sentences: {len(sentences)}")

Number of sentences: 676869


## Load Pickle

In [5]:
# # src will be sentences_periodika.pkl in pickles folder
# src = Path("../pickles/sentences_periodika.pkl")
# # assert that the file exists
# assert src.exists(), f"{src} does not exist"
# # read pickle
# with open(src, "rb") as f:
#     sentences = pickle.load(f)
# # how many sentences do we have?
# print(f"Number of sentences: {len(sentences)}")



Number of sentences: 9767880


In [8]:
# sentence lengths
sentence_lengths = [len(sentence) for sentence in sentences]
# min, avg and max length of sentences
min_length = min(sentence_lengths)
avg_length = sum(sentence_lengths) / len(sentences)
max_length = max(sentence_lengths)
print(f"Min sentence length: {min_length}")
print(f"Avg sentence length: {avg_length}")
print(f"Max sentence length: {max_length}")

Min sentence length: 2
Avg sentence length: 44.00467446433505
Max sentence length: 553


In [9]:
# lets do some extra logging using a callback
# https://datascience.stackexchange.com/questions/9819/number-of-epochs-in-gensim-word2vec-implementation

from gensim.models.callbacks import CallbackAny2Vec



class LossLogger(CallbackAny2Vec):
    '''Output loss at each epoch'''
    def __init__(self):
        self.epoch = 1
        self.start_losses = []
        self.end_losses = []

    def on_epoch_begin(self, model):
        loss = model.get_latest_training_loss()
        print(f'Epoch: {self.epoch} - START loss {loss}')
        self.start_losses.append(loss)

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print(f'Epoch {self.epoch} - END  Loss: {loss}')
        self.end_losses.append(loss)
        self.epoch += 1

In [10]:
# Your model params:
loss_logger = LossLogger()
CONTEXT_WINDOW = 5
NEGATIVES = 5
MIN_COUNT = 5
EPOCHS = 20
sentences = sentences
# start time
start = datetime.now()
print(f"Start time: {start}")
model = gensim.models.word2vec.Word2Vec(sentences=sentences,
                                      sg=1,
                                      window=CONTEXT_WINDOW,
                                      negative=NEGATIVES,
                                      min_count=MIN_COUNT,
                                      callbacks=[loss_logger],
                                      compute_loss=True,
                                      epochs=EPOCHS)
# end time
end = datetime.now()
print(f"End time: {end}")
# save model
model.save("../models/word2vec_latsenrom.model")

2025-02-07 11:29:46,057 : INFO : collecting all words and their counts
2025-02-07 11:29:46,058 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2025-02-07 11:29:46,132 : INFO : PROGRESS: at sentence #10000, processed 404960 words, keeping 53341 word types
2025-02-07 11:29:46,203 : INFO : PROGRESS: at sentence #20000, processed 919455 words, keeping 78732 word types


Start time: 2025-02-07 11:29:46.056113


2025-02-07 11:29:46,283 : INFO : PROGRESS: at sentence #30000, processed 1364882 words, keeping 97160 word types
2025-02-07 11:29:46,339 : INFO : PROGRESS: at sentence #40000, processed 1618759 words, keeping 117713 word types
2025-02-07 11:29:46,391 : INFO : PROGRESS: at sentence #50000, processed 1945606 words, keeping 128202 word types
2025-02-07 11:29:46,461 : INFO : PROGRESS: at sentence #60000, processed 2372376 words, keeping 137170 word types
2025-02-07 11:29:46,518 : INFO : PROGRESS: at sentence #70000, processed 2686281 words, keeping 147298 word types
2025-02-07 11:29:46,671 : INFO : PROGRESS: at sentence #80000, processed 3499235 words, keeping 184174 word types
2025-02-07 11:29:46,778 : INFO : PROGRESS: at sentence #90000, processed 4208269 words, keeping 200358 word types
2025-02-07 11:29:46,838 : INFO : PROGRESS: at sentence #100000, processed 4585048 words, keeping 214584 word types
2025-02-07 11:29:46,905 : INFO : PROGRESS: at sentence #110000, processed 5004970 words,

Epoch: 1 - START loss 0.0


2025-02-07 11:29:54,882 : INFO : EPOCH 0 - PROGRESS: at 1.58% examples, 328881 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:29:55,931 : INFO : EPOCH 0 - PROGRESS: at 2.90% examples, 332301 words/s, in_qsize 6, out_qsize 0
2025-02-07 11:29:56,950 : INFO : EPOCH 0 - PROGRESS: at 3.98% examples, 332426 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:29:57,948 : INFO : EPOCH 0 - PROGRESS: at 6.25% examples, 338527 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:29:58,968 : INFO : EPOCH 0 - PROGRESS: at 8.25% examples, 339139 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:30:00,017 : INFO : EPOCH 0 - PROGRESS: at 9.98% examples, 339801 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:30:01,034 : INFO : EPOCH 0 - PROGRESS: at 11.27% examples, 338684 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:30:02,054 : INFO : EPOCH 0 - PROGRESS: at 11.69% examples, 336754 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:30:03,083 : INFO : EPOCH 0 - PROGRESS: at 12.51% examples, 336549 words/s, in_qsize 5, o

...

2025-02-07 11:50:47,545 : INFO : EPOCH 18 - PROGRESS: at 1.60% examples, 339939 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:50:48,584 : INFO : EPOCH 18 - PROGRESS: at 2.92% examples, 343768 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:50:49,633 : INFO : EPOCH 18 - PROGRESS: at 4.14% examples, 339694 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:50:50,688 : INFO : EPOCH 18 - PROGRESS: at 6.62% examples, 346788 words/s, in_qsize 6, out_qsize 0
2025-02-07 11:50:51,719 : INFO : EPOCH 18 - PROGRESS: at 8.53% examples, 349671 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:50:52,730 : INFO : EPOCH 18 - PROGRESS: at 10.36% examples, 352077 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:50:53,738 : INFO : EPOCH 18 - PROGRESS: at 11.36% examples, 352362 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:50:54,762 : INFO : EPOCH 18 - PROGRESS: at 11.95% examples, 350783 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:50:55,768 : INFO : EPOCH 18 - PROGRESS: at 12.75% examples, 350335 words/s, in_

Epoch 19 - END  Loss: 104406640.0
Epoch: 20 - START loss 104406640.0


2025-02-07 11:51:56,037 : INFO : EPOCH 19 - PROGRESS: at 1.62% examples, 347015 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:51:57,060 : INFO : EPOCH 19 - PROGRESS: at 2.92% examples, 345831 words/s, in_qsize 6, out_qsize 0
2025-02-07 11:51:58,072 : INFO : EPOCH 19 - PROGRESS: at 4.12% examples, 342295 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:51:59,087 : INFO : EPOCH 19 - PROGRESS: at 6.41% examples, 346663 words/s, in_qsize 6, out_qsize 0
2025-02-07 11:52:00,103 : INFO : EPOCH 19 - PROGRESS: at 8.38% examples, 348869 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:52:01,120 : INFO : EPOCH 19 - PROGRESS: at 10.22% examples, 351224 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:52:02,169 : INFO : EPOCH 19 - PROGRESS: at 11.33% examples, 349782 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:52:03,190 : INFO : EPOCH 19 - PROGRESS: at 11.88% examples, 348563 words/s, in_qsize 5, out_qsize 0
2025-02-07 11:52:04,201 : INFO : EPOCH 19 - PROGRESS: at 12.70% examples, 348260 words/s, in_

Epoch 20 - END  Loss: 106169448.0
End time: 2025-02-07 11:53:02.743469


2025-02-07 11:53:02,973 : INFO : not storing attribute cum_table
2025-02-07 11:53:03,439 : INFO : saved ../models/word2vec_latsenrom.model
